## nomad-parser-nexus demo for Atom Probe Microscopy (APM) data

### **Step -1:** Create a new virtual environment and set up dependencies for jupyterlab_h5web.<br> This section **should only be run if you do not have Jupyter Lab, its extensions and the extra needed packages.** <br>  For use within the Nomad UI this section should be skipped.<br> These cells can be run in a Jupyter notebook or copied to a terminal without the "!".

Run the following to start with a fresh virtualenv in **your terminal** in which you are installing and then running **Jupyter lab** if you wish to: <br>
```pip install virtualenv && virtualenv --python=python3.7 .nexusenv && source .nexusenv/bin/activate```

Install jupyter, jupyter-lab and h5web extensions.

In [ ]:
! pip install --upgrade nodejs && pip install ipywidgets h5py==3.5.0 h5glance==0.7 h5grove==0.0.14 jupyterlab[full]==3.2.9 jupyterlab_h5web[full]==1.3.0 punx==0.2.5 nexpy==0.14.1 silx[full]==1.0.0 && jupyter lab build

Enable the extensions

In [ ]:
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
! jupyter serverextension enable jupyterlab_h5web

### **Step 0:** Installing and testing nomad-parser-nexus module. <br> This section **should only be run if you are not running this within NOMAD.** 

Install nomad and its dependencies. Do not run the following cell if you have a nomad installation running. 

In [ ]:
! pip install --upgrade pip && pip install nomad-lab==1.0.0 --extra-index-url https://gitlab.mpcdf.mpg.de/api/v4/projects/2187/packages/pypi/simple
! pip list

Install the nexusparser and its requirements

In [ ]:
! git clone https://github.com/nomad-coe/nomad-parser-nexus.git --recursive && cd nomad-parser-nexus && git status && pip install -r requirements.txt && pip install -e .[all]

The following cell restarts the kernel after the nexusparser installation

In [ ]:
import os
os._exit(00)

### **Step 1:** Check that everything is ready to go and download example data for APM. 

Check dependencies of nomad and of the nexusparser and prints them. If the nexusparser and nomad-lab are installed, you are ready to go.<br> Check if jupyterlab_h5web server and lab extensions are enabled and OK. 

In [ ]:
! pip list | grep 'nomad\|nexus'
! jupyter serverextension list
! jupyter labextension list

For this case, the data can be found in zenodo.

In [ ]:
import shutil  # unpacks in current path unless an additional path argument is provided
# http://dx.doi.org/10.5281/zenodo.5911240
! curl --output APM.LEAP.Datasets.1.zip https://zenodo.org/record/5911240/files/APM.LEAP.Datasets.1.zip
shutil.unpack_archive('APM.LEAP.Datasets.1.zip')
! curl --output APM.LEAP.Datasets.2.zip https://zenodo.org/record/5911240/files/APM.LEAP.Datasets.2.zip
shutil.unpack_archive('APM.LEAP.Datasets.2.zip')
! curl --output APM.LEAP.Datasets.3.zip https://zenodo.org/record/5911240/files/APM.LEAP.Datasets.3.zip
shutil.unpack_archive('APM.LEAP.Datasets.3.zip')
# remove obsolete example files
! rm -f R31_06365-v02.ELabFTW.12.json

These files should serve exclusively as examples. Use always a triplet of files (for now), i.e. an (apt or pos or epos) plus an (rng or rrng), and the json file respectively.

### **Step 2:** Run your APM-specific dataconverter/readers/ on the example data.

Set the nexusparser directory.

In [ ]:
import os
import nexusparser
nexus_dir = os.path.dirname(nexusparser.__file__)  # where the nexusparser module is located!!!!
print(nexus_dir)

Now we run our parser. The --reader flag takes the atom probe microscopy reader (apm), the --nxdl flag takes the application definition for this technique.<br> 

Inspect what can/should be in the NeXus file.

In [ ]:
# to inspect what can/should all be in the NeXus file
! python3 {nexus_dir}/tools/dataconverter/convert.py \
--reader apm \
--nxdl NXapm \
--generate-template

Parse the files to convert them to an HDF5 file.

In [ ]:
! python3 {nexus_dir}/tools/dataconverter/convert.py \
--reader apm \
--nxdl NXapm \
--input-file R31_06365-v02.pos \
--input-file R31_06365-v02.rrng \
--input-file ManuallyCollectedMetadata.json \
--output apm0002.test.nxs

The key take home message is that the command above-specified triggers the automatic creation of the HDF5 file.<br>
This *.nxs file, is an HDF5 file.

### **Step 3:** Inspect the HDF5/NeXus file apm*.test.nxs using H5Web.

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
h5_file_name = 'apm0002.test.nxs'

In [ ]:
H5Web(h5_file_name)

You can also visualize the .nxs file by double clikcing it within Jupyter lab. 

### **Step 4:** Do some post-processing with the generated apm0002.test.nxs file.

To compute a mass-to-charge histogram and explore eventual ranging definitions that have also been carried over in the conversion step (step 6).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 300
import h5py as h5
#needs shutils for decompressing zip archives, which is a default module/package in Python since >=v3.6

Read mass-to-charge-state ratio values, create a histogram ("mass spectrum"), and mark ranges.

In [ ]:
# load data and ranges
hf = h5.File(h5_file_name, 'r')
mq = hf['entry/atom_probe/mass_to_charge_conversion/mass_to_charge'][:]
nions = np.uint32(hf['entry/atom_probe/ranging/number_of_iontypes'])
print('Array with mass-to-charge-state ratios loaded')
print(str(nions) + ' iontypes were distinguished')

In [ ]:
# define binning
[mqmin, mqmax] = [0., 100.0]  # Da np.max(mq)]
print('Dataset ranging from [' + str(mqmin) + ', ' + str(mqmax) +'] Da.')
mqincr = 0.01 #Da
print('Using a mass-to-charge-state ratio resolution of '+str(mqincr)+' Da.')

In [ ]:
# transform collection of mass-to-charge-state ratios into a histogram
hst1d = np.unique(np.uint64(np.floor((mq[np.logical_and(mq >= mqmin, mq <= mqmax)] - mqmin) / mqincr)), return_counts=True)
nbins = np.uint64((mqmax - mqmin) / mqincr + 1)
print('Histogram has ' + str(nbins) + ' bins.')

In [ ]:
# use matplotlib and numpy to plot histogram data 
xy = np.zeros([nbins, 2], np.float64)
xy[:,0] = np.linspace(mqmin + mqincr, mqmax + mqincr, nbins, endpoint=True)
xy[:,1] = 0.5  # * np.ones([nbins], np.float64)  # 0.5 to be able to plot logarithm you can not measure half an atom
for i in np.arange(0, len(hst1d[0])):
    binidx = hst1d[0][i]
    xy[binidx, 1] = hst1d[1][i]
print('Mass-to-charge-state histogram created.')

In [ ]:
[xmi, xmx, ymi, ymx] = [mqmin, 10**np.ceil(np.log10(mqmax)), 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]
[xmi, xmx, ymi, ymx] = [mqmin, mqmax, 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]
fig, cnts_over_mq = plt.subplots(1, 1)
plt.plot(xy[:, 0], xy[:, 1], color='blue', alpha=0.5, linewidth=1.0)
for i in np.arange(1,nions + 1):
    # load ranges and plot them
    ranges = hf['entry/atom_probe/ranging/peak_identification/ion' + str(i) + '/mass_to_charge_range'][:]
    for min_max in ranges:
        cnts_over_mq.vlines(min_max[0], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color='grey', linestyles='dotted')
        cnts_over_mq.vlines(min_max[1], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color='grey', linestyles='dotted')
        #rng = Rectangle((min_max[0], ymi), min_max[1] - min_max[0], ymx - ymi, edgecolor='r', facecolor="none")
# plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9], ['Min', '0.0025', '0.025', '0.25', '0.50', '0.75', '0.975', '0.9975', 'Max'])
plt.yscale('log')
plt.legend( [r'Mass-to-charge-state ratio $\Delta\frac{m}{q} = $'+str(mqincr)+' Da'], loc='upper right')
plt.xlabel(r'Mass-to-charge-state-ratio (Da)')
plt.ylabel(r'Counts')
print('Mass-to-charge-state histogram visualized.')
# scale bar with add margin to the bottom and top of the yaxis to avoid that lines fall on x axis
margin=0.01  # polishing the margins
plt.xlim([-margin * (xmx - xmi) + xmi, +margin * (xmx - xmi) + xmx])
plt.ylim([ymi, +margin * (ymx - ymi) + ymx])

In [ ]:
#save the figure
figfn = h5_file_name + '.MassToChargeStateRatios.png'
fig.savefig(figfn, dpi=300, facecolor='w', edgecolor='w', orientation='landscape', format='png', 
            transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)
#plt.close('all')
print(figfn + ' stored to disk.')

### Further comments:

* Feel free to explore our paraprobe container in the north branch for more advanced processing
* Add legends and names for iontypes